# Analysis of the National Survey of Health

## Jason Piccone, Ph.D.

## Part II: Basic Analysis

# Import Libraries

In [ ]:
from collections import Counter
from matplotlib import style 
from sklearn import linear_model, decomposition
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_curve, auc, confusion_matrix, roc_auc_score, mean_squared_error
from sklearn.pipeline import Pipeline

import csv
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random

# Parameters

In [ ]:
path = "/national survey on drug use and health 2012/ICPSR_34933/DS0001/"
d_file = 'combined_df.csv'
style.use('ggplot')

rand_state = 42
trees = 100 # number of trees in the random forest


# Read Data

In [ ]:
df = pd.read_csv(path+d_file)

print(df.shape)
print(df.head())

# Explore target distributions

In [ ]:
def dv_plots(df):

    #Respondent been arrested?
    counts = Counter(df.BOOKED).values()
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    objects = ('Yes','No')
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, counts, align='center', alpha=0.5, width=.5)
    ax1.set_xticks([0,1])    
    ax1.set_xticklabels(objects, rotation=0, fontsize=13)
    ax1.set_title('Have You Ever Been Arrested?')
    plt.show()

    # obviously far more people have not been booked, but this is still a fine variable
    # to test on, as many cases include variable values that are infrequent

    #Respondent overall health
    ax=plt.subplot(111)
    plt.hist(df.HEALTH, bins=5, alpha=0.5)  
    ax.set_xticklabels(('Excellent', 'Very Good', 'Good','Fair', 'Poor'), fontsize=8)
    ax.set_xticks([-0.80,0.10,0.90,1.80,2.70])
    plt.title('Distribution of Overall Health Ratings')
    plt.xlabel("Health Rating", fontsize=16)  
    plt.ylabel("Count", fontsize=16)
    plt.show()

    # positively skewed - most people are relatively healthy, with a few trailing off
    # to fair and poor values

dv_plots(df)

# Explore feature distributions

In [ ]:
def plot_features(df):

    '''explore demographic information:'''
    #Respondent Sex
    counts = Counter(df.IRSEX).values()
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    objects = ('Male','Female')
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, counts, align='center', alpha=0.5, width=.5)
    ax1.set_xticks([0,1])    
    ax1.set_xticklabels(objects, rotation=0, fontsize=13)
    ax1.set_title('What is Your Gender?')
    plt.show()

    #Respondent marital status
    cMarried = list(Counter(df.MARRIED).values())[1]
    cDivorced = list(Counter(df.DIVORCED).values())[1]
    cNeverM = list(Counter(df.NEVER_MARRIED).values())[1]
    tMARRIED = [cMarried,cDivorced,cNeverM]
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    objects = ('Married','Divorced','Never Married')
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, tMARRIED, align='center', alpha=0.5, width=.5)
    ax1.set_xticks([0,1,2,3])    
    ax1.set_xticklabels(objects, rotation=0, fontsize=13)
    ax1.set_title('What is Your Marital Status?')
    plt.show()

    #Respondent county size
    cLarge = list(Counter(df.COUNTY_LARGE).values())[1]
    cMedium = list(Counter(df.COUNTY_SMALL).values())[1]
    cSmall = list(Counter(df.COUNTY_NONMETRO).values())[1]
    tMETRO = [cLarge,cMedium,cSmall]
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    objects = ('Large','Medium','Small')
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, tMETRO, align='center', alpha=0.5, width=.5)
    ax1.set_xticks([0,1,2])    
    ax1.set_xticklabels(objects, rotation=0, fontsize=13)
    ax1.set_title('What Type of County do you Live in?')
    plt.show()

    #Respondent employment status
    cFull = list(Counter(df.FULLTIME).values())[1]
    cPart = list(Counter(df.PARTTIME).values())[1]
    cUne = list(Counter(df.UNEMPLOYED).values())[1]
    cOther = list(Counter(df.OTHER).values())[1]
    tEMPLOY = [cFull, cPart, cUne, cOther]
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    objects = ('Full-Time','Part-Time','Unemployed','Other')
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, tEMPLOY, align='center', alpha=0.5, width=.5)
    ax1.set_xticks([0,1,2,3])    
    ax1.set_xticklabels(objects, rotation=0, fontsize=13)
    ax1.set_title('What Is Your Employment Status?')
    plt.show()

    #Respondent school status
    cSCHOOL = list(Counter(df.SCHENRL).values())
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    objects = ('Yes','No')
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, cSCHOOL, align='center', alpha=0.5, width=.5)
    ax1.set_xticks([0,1])    
    ax1.set_xticklabels(objects, rotation=0, fontsize=13)
    ax1.set_title('Are You Currently Enrolled in Any School?')
    plt.show()

    #Respondent income
    ax=plt.subplot(111)
    plt.hist(df.IRPINC3, bins=7, alpha=0.5)  
    ax.set_xticklabels(('< 10k', '10-19k','20-29k','30-39k','40-49k','50-75k','75k+'), fontsize=8)
    ax.set_xticks([-0.40,0.10,0.60,1.085,1.60,2.10,2.60])
    plt.title("Respondent's Income Level")
    plt.show()
    

plot_features(df)



In [ ]:
# function to partition the data
def split_data(df,y):
        # param df = the dataframe to partition
        # param y = one of the two possible target variables

        '''divide sample into train and test'''      
        y = df[y]
        X = df.drop(['BOOKED','HEALTH'], axis=1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=rand_state)

        return(X_train, X_test, y_train, y_test)
    
    
def run_rf(df,y):
    
    # param df: dataframe
    # param y: the target variable (either 'BOOKED' or 'HEALTH')


    if y == 'BOOKED':
        rf = RandomForestClassifier(n_estimators=trees)
    else:
        rf = RandomForestRegressor(n_estimators=trees)
            
    # collect the data partitions
    X_train, X_test, y_train, y_test = split_data(df,y)


    # run the random forest model
    def run_rf(X_train, X_test, y_train, y_test):

        rf.fit(X_train, y_train)

        # assess model accuracy for 'booked' 
        if y == 'BOOKED':
            
            disbursed = rf.predict_proba(X_train)

            fpr, tpr, thresholds = metrics.roc_curve(y_train, disbursed[:,1])
            print('Random forest predicting Booked AUC (train set) = ' + str(metrics.auc(fpr, tpr)))

            #plot auc
            plt.plot(fpr,tpr)
            plt.xlabel('False Positives')
            plt.ylabel('True Positives')
            plt.title('AUC: Have You Ever Been Arrested-Training Set')
            plt.show()

            #how accurate on the test set
            disbursed = rf.predict_proba(X_test)
            fpr, tpr, thresholds = metrics.roc_curve(y_test, disbursed[:,1])
            print('Random forest predicting Booked AUC (test set) = ' + str(metrics.auc(fpr, tpr)))

            #plot auc
            plt.plot(fpr,tpr)
            plt.xlabel('False Positives')
            plt.ylabel('True Positives')
            plt.title('AUC: Have You Ever Been Arrested-Test Set')
            plt.show()

            disbursed = rf.predict(X_test)     
            print('test set confusing matrix:')
            print(confusion_matrix(disbursed, y_test))

            def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
                plt.imshow(cm, interpolation='nearest', cmap=cmap)
                plt.title(title)
                plt.colorbar()
                plt.tight_layout()
                plt.ylabel('True label')
                plt.xlabel('Predicted label')
                plt.show()


            cm = confusion_matrix(y_test, disbursed)
            np.set_printoptions(precision=2)
            print('Confusion matrix, without normalization')
            print(cm)
            plt.figure()
            plot_confusion_matrix(cm)
            '''we can see we predict people who have not been booked very well, but we also predict
            too many people to not be booked (quite a few false negatives). '''


            # Normalize the confusion matrix by row (i.e by the number of samples
            # in each class)
            cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            print('Normalized confusion matrix')
            print(cm_normalized)
            plt.figure()
            plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')
         
        # assess model accuracy for 'health' 
        else:
            p1 = rf.predict(X_test)
            mse = mean_squared_error(y_test, p1)
            print('root mean squared error = '+str(math.sqrt(mse)))
            

        '''variable importance for the RF'''
        importances = rf.feature_importances_
        std = np.std([tree.feature_importances_ for tree in rf.estimators_],
                     axis=0)
        indices = np.argsort(importances)[::-1]

        # plot the top 10 features
        top10 = sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), X_train.columns), 
                     reverse=True)[:10]
        top10 = pd.DataFrame(top10)

        objects = top10[1]
        plt.bar(range(top10.shape[0]), top10[0], align="center", alpha=0.5)
        plt.xticks(range(10), objects, rotation='vertical')
        plt.title('Feature Importance Predicting {0}'.format(y))
        plt.show()


    run_rf(X_train, X_test, y_train, y_test)


In [ ]:
# run random forest classifier on 'have you ever been booked?'
run_rf(df,'BOOKED')



In [ ]:
# run random forest regressor on 'how is your overall health?'
run_rf(df,'HEALTH')

# PCA Analysis

One alternative is to perform a principle component analysis (PCA) to combine features. This
will be especially effective if the components that emerge are actually interpretable, which
is not gaurantted! 

We could also explore these variables in greater detail. For example, TXEVER (ever received
treatment for drugs/alcohol) could be further explored with the feature AUMOTVYR,
which is what prompted people to get treatment for past mental health issues -- such as whether
they did so voluntarily or not. But it's unclear if people who were forced to receive 
mental health treatment where treated for drug/alcohol abuse or not.



In [ ]:

def booked(df):
    
    X_train, X_test, y_train, y_test = split_data(df,'BOOKED')


    #How many components is ideal? Let's first test how much variance is accounted for:
    # Plot the PCA spectrum
    pca = decomposition.PCA()
    pca.fit(X_train)
    plt.figure(1, figsize=(4, 3))
    plt.clf()
    plt.axes([.2, .2, .7, .7])
    plt.plot(pca.explained_variance_, linewidth=2)
    plt.title('Components by Variance Explained', fontsize=13)
    plt.xlabel('Number of Components')
    plt.ylabel('Explained Variance')
    plt.show()

    #After 10 components, each additional component explains less than .5% of the total variance.
    #However, the total variance explained increases up to 50 components. We can try both.
    
    # fits PCA, transforms data and fits the decision tree classifier
    # on the transformed data
    pipe = Pipeline([('pca', PCA(n_components=40)),
                     ('tree', RandomForestClassifier(n_estimators=100))])
    pipe.fit(X_train, y_train)

    disbursed = pipe.predict_proba(X_test)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, disbursed[:,1])
    print('New AUC = ' + str(metrics.auc(fpr, tpr))) 

booked(df)

In [ ]:
# trying the pca analysis for the overall health outcome:
    
def pca_health(df):
    
    X_train, X_test, y_train, y_test = split_data(df,'HEALTH')


    pipe = Pipeline([('pca', PCA(n_components=40)),
                     ('tree', RandomForestRegressor(n_estimators=trees))])
    pipe.fit(X_train, y_train)

    disbursed = pipe.predict(X_test)
    mse = mean_squared_error(y_test, disbursed)
    #.834
    print('root mean squared error = ' + str(math.sqrt(mse)))

    
pca_health(df)

"""
theoretically using PCA could improve the model's predictive ability as it may
reduced overfitting - by reducing the number of features in an orthogonal manner,
we were able to remove variables which just added noise to the train model - so the random
forest trained on that noise in the benchmark model. In addition, we gain power through
the reduction of dimensionality. Nevertheless, PCA depricated the model slightly
"""

In [ ]:
df.to_csv(path+'df2.csv', index=False)  
